In [1]:
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import os
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from torchtext.vocab import Vocab, GloVe
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torchtext.data import get_tokenizer


In [52]:
def data_cleaning(text):
    # 大小写
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace("#" , " ")
    text = text.replace("." , "")
    text = text.replace("%", "")




    text = re.sub('https?://[A-Za-z0-9./]+', '', text)
    text = re.sub('http?://[A-Za-z0-9./]+', '', text)
    text = re.sub('www.[A-Za-z0-9./]+', '', text)
    text = re.sub("\d+", "", text)


    encoded_string = text.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    return decode_string

In [53]:
global_vectors = GloVe(name='twitter.27B', dim=200)

In [54]:
tokenizer = get_tokenizer("basic_english") ## We'll use tokenizer available from PyTorch
embeddings = global_vectors.get_vecs_by_tokens(tokenizer("fuccccccccccck"), lower_case_backup=True)
s1 =  embeddings.tolist()
embeddings1 = global_vectors.get_vecs_by_tokens(tokenizer("fuck"), lower_case_backup=True)
s2 = embeddings1.tolist()

In [55]:
s1, s2

([[0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,


In [56]:
data_folder_path = '/mnt/a/OneDrive/UNSW/COMP9444/9444_toxic_comment_classification/data/'
train = pd.read_csv(data_folder_path + 'train.csv')
test_data = pd.read_csv(data_folder_path + 'test.csv')
test_labels = pd.read_csv(data_folder_path + 'test_labels.csv')
test_data = pd.concat([test_data, test_labels], axis=1)

classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train[train[classes[1]] == 1].sample(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
135109,d2b6d3324dc09a60,ARSE YOU FUCKING PRICK. FUCK OFF.,1,1,1,0,1,0
75389,c9b0eba0f9d55fbc,P.S Im a fucking cunt rag and enjoy the taste ...,1,1,1,0,0,0
92124,f64b7d5518c05c76,bitch \n\nfuck you b***h,1,1,1,0,1,0
27450,48a6c30c8ed77d59,FUCK YOU FUCK YOU FUCK YOU \n\nFFFFFFFFUUUUUUU...,1,1,1,0,1,0
76433,cca06f093aec503b,FUCK QUEBECFAGGOTS. FUCKING SEPERATE SO WE CAN...,1,1,1,0,1,0
38775,677b5c663a0758ae,Hi \n\nHi you are gay sad fucking old nerd tha...,1,1,1,0,1,1
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
120395,83f66f6c00cf2985,DONT SEND ME A WARNING AGAIN OR I WILL RAPE YO...,1,1,1,1,0,0
8619,16e110672dde00d9,How civil is this - who the fuck do you think ...,1,1,1,0,1,0
139493,eaa76f9c074e3aa5,Daedalus is nothing but a filthy n!gger and a ...,1,1,1,0,1,1


In [57]:
#seed = 42
#train, test_data = train_test_split(train, test_size=0.2, random_state=seed)


In [58]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [59]:
for cls in classes:
    rate = train[cls].sum() / len(train)
    rate = np.round(rate*100, 3)
    print(cls +' rate: ', rate, "%")

toxic rate:  9.584 %
severe_toxic rate:  1.0 %
obscene rate:  5.295 %
threat rate:  0.3 %
insult rate:  4.936 %
identity_hate rate:  0.88 %


In [60]:
for cls in classes:
    rate = test_data[cls].sum() / len(test_data)
    rate = np.round(rate*100, 3)
    print(cls +' rate: ', rate, "%")

toxic rate:  -54.253 %
severe_toxic rate:  -57.989 %
obscene rate:  -55.819 %
threat rate:  -58.091 %
insult rate:  -55.992 %
identity_hate rate:  -57.764 %


In [61]:
print(type(train))

<class 'pandas.core.frame.DataFrame'>


In [62]:
#l_train = []
l_train_drop =[]

for idx in range(0, len(train)):
    comment = train["comment_text"].iloc[idx]
    clean = data_cleaning(comment)
    token = tokenizer(clean)
    if len(token) == 0:
        l_train_drop.append(idx)
        continue
    train["comment_text"].iloc[idx] = global_vectors.get_vecs_by_tokens(token, lower_case_backup=True)


#    row['embeddings'] = embeddings

/tmp/ipykernel_8331/1009963608.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["comment_text"].iloc[idx] = global_vectors.get_vecs_by_tokens(token, lower_case_backup=True)


In [63]:
train.drop(l_train_drop, axis = 0, inplace = True)

In [64]:
len(train)

159566

In [70]:
for idx in range(0, len(train)):

20

In [85]:
from torch.nn.utils.rnn import pad_sequence, pack_sequence

for idx in range(0, 5):
    comment = train["comment_text"][idx]
    print(comment)
    pad = pack_sequence(comment)
    print(pad)


tensor([[-0.1937, -0.1251, -0.5924,  ...,  0.7811, -0.1109, -0.2980],
        [ 0.2539,  0.3569,  0.2210,  ...,  0.1067, -0.1587,  0.1287],
        [ 0.4935,  0.3570,  0.6607,  ...,  0.1771, -0.5369, -0.2970],
        ...,
        [ 0.1430, -0.4178,  0.2529,  ..., -0.7805,  0.3792,  0.4934],
        [ 0.1588,  0.2819, -0.4703,  ...,  0.8744, -0.5519, -0.1927],
        [-0.0218, -0.0115, -0.4395,  ...,  0.4765,  0.2968,  0.6029]])
PackedSequence(data=tensor([-0.1937,  0.2539,  0.4935,  ...,  0.4934, -0.1927,  0.6029]), batch_sizes=tensor([51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 

In [ ]:
pad.data.shape

In [90]:
comment.shape

torch.Size([18, 200])

In [91]:
padded = pack_sequence(train["comment_text"])

RuntimeError: [enforce fail at alloc_cpu.cpp:66] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 631626054400 bytes. Error code 12 (Cannot allocate memory)